# webpack 之 jquery-mousewheel 引入事件

在项目中引入 jquery-mousewheel 的正确步骤。
知识点： 

1. webpack
2. imports-loader

## jquery-mousewheel tutorial 
本 tutorial 的功能是想项目使用基于 jquery 插件的 jqury-mousewheel 插件的实现。
## 创建项目
### 1. 初始化目录
```bash
mkdir mousewheel-tutorial
cd mousewheel-tutorial
npm init
```

### 2. 安装相关依赖库

1. 安装 webpack
```bash
npm i -D webpack
```
2. 配置 webpack 在项目根目录下添加一个 webpack.config.js, 目的主要是对对项目 js 编译的配置。
```javascript
module.exports = {
	entry: 'index.js',
	output: {
		path: './dist', // 不存在的话会自动创建。
		filename: 'bundle.js'
	}
}
```
3. 执行 webpack 测试
```bash
webpack
```
```
Hash: 85616124b891b861794c
Version: webpack 2.2.1
Time: 59ms
    Asset     Size  Chunks             Chunk Names
bundle.js  2.67 kB       0  [emitted]  main
   [0] ./index.js 42 bytes {0} [built]
```
4. 安装 jquery, jquery-mousewheel
```
npm i -S jqeury jquery-mousewheel
```

### 3. AMD 方式引入 `jquery-mousewheel`

1. 修改 index.js 为

```javascript
console.log([1, 2, 3, 4].map(e => e ** e))
require('jquery-mousewheel') // jquery-mousewheel 是依赖于 jquery 的，我们故意把 jquery-mousewheel 写在 jquery 的前面。
const $ = require('jquery')
console.log($('#content-01').mousewheel(function (event) {
	console.log('event:', event)
}))
```

2. `webpack --display-reasons --colors` 编译结果

<img src="./mousewheel-tutorial/buildWithAmd.jpeg">

- amd : 表示 AMD 方式引入
- cjs : 表示 CommonJS 方式引入

通过加上 `--display-reasons` 可以看出 [0] 之所以先加载 jquery 是因为在 `jquery.mousewheel.js 12:8-35` 使用 amd 依赖了 jquery.
其代码片段如下：

```javascript
(function (factory) {
    debugger; // 在这里加上 debugger 可以打个断点看出分支第一个判断是永真的。
    if ( typeof define === 'function' && define.amd ) {
        // AMD. Register as an anonymous module.
        define(['jquery'], factory);
    } else if (typeof exports === 'object') {
        // Node/CommonJS style for Browserify
        module.exports = factory;
    } else {
        // Browser globals
        factory(jQuery);
    }
}(function ($) {// mousewheel 插件扩展到 jQuery 上的代码 })
```

从 if 的注释可以看到：`//ADM. Register as an anonymouse module` AMD 的 define 竟然自然注册到项目中了，所以在使用 `require('jquery-mousewheel')` 时会执行第一个条件分支。所以在编译时也会有 amd 的方法引入。通过在`jqury-mousewheel`中打个断点可以看出 webpack 在编译时对第一个条件竟然是永真的。

<img src="./mousewheel-tutorial/ifTrue.jpeg">

如果我们不想使用 ADM 方式怎么办？

### 4. CommonJS (cjs) 方式来引入 

如果不想使用 AMD 而使用 CommonJS 来引入的话，可以使用 [imports-loader](https://github.com/webpack-contrib/imports-loader) 来完成这一功能。
1. 安装 imports-loader

```
npm i -D imports-loader
```

2. 在 index.js 中把 `require('jquery-mousewheel')` 改成 ` var mousewheel = require('imports?define=>false!jquery-mousewheel')`
```
console.log([1, 2, 3, 4].map(e => e ** e))
const $ = require('jquery')
var mousewheel = require('imports-loader?define=>false!jquery-mousewheel') 
mousewheel($) // 换成 CommonJS 方式要自己调用返回的工厂方法 factory.
console.log($('#content-01').mousewheel(function (event) {
	console.log('event:', event)
}))
```

3. 编译

可以看出所有的模块都有 CommonJS 来加载了

<img src="./mousewheel-tutorial/buildWithCjs.jpeg">